In [57]:
import requests
import os
from dotenv import load_dotenv
import json
import polars as pl
import io
import polars.selectors as cs

# Load environment variables
load_dotenv()

# Get API token from environment
LAMA_API_TOKEN = os.getenv('SECRET_LAMA_API_TOKEN')
BASE_URL = "https://app.lamapoll.de/api/v2"

# Set up headers for authentication
headers = {
    "Authorization": f"Bearer {LAMA_API_TOKEN}",
    "Content-Type": "application/json"
}

## Available Polls

In [3]:
response = requests.get(f"{BASE_URL}/polls", headers=headers)
print(f"Status Code: {response.status_code}")
response.raise_for_status()
polls = response.json()
print(f"Number of polls: {len(polls) if isinstance(polls, list) else 'Unknown'}")
poll_id = polls[0]["id"]
print(poll_id)

Status Code: 200
Number of polls: 2
1850964


## Poll Details

In [4]:

response = requests.get(f"{BASE_URL}/polls/{poll_id}", headers=headers)
response.raise_for_status()
poll_details = response.json()

In [5]:
poll_details["url"]

'https://caritas.sslsurvey.de/U25-Peerbefragung'

In [6]:
poll_details["settings"]["access"]

{'isPublic': True,
 'isCookieLocksEnabled': False,
 'isIpLocksEnabled': False,
 'isCorrectionModeEnabled': False,
 'ipWhitelist': ['*']}

In [7]:
poll_details["state"]

{'responsesCount': 86,
 'isRunning': True,
 'lastActivityAt': '2025-10-06T12:31:12+00:00'}

In [8]:
poll_details["pagesCount"]

3

## Questions

In [9]:
response = requests.get(f"{BASE_URL}/polls/{poll_id}/questions", headers=headers)
response.raise_for_status()
questions = response.json()
len(questions)

22

In [10]:
questions[0]

{'id': 27937500,
 'pollId': 1850964,
 'pageId': 8134161,
 'pagePosition': 0,
 'position': 2,
 'question': {'de': 'Welche drei Worte verbindest du spontan mit [U25]?\xa0'},
 'description': {},
 'anonym': False,
 'mandatory': False,
 'type': 'INPUT',
 'groups': [{'name': {'de': 'Erstes Textfeld:'},
   'labels': [],
   'range': [],
   'varnames': ['V1'],
   'codes': [],
   'items': [{'id': 'V01_8ff5376a05a639fce2327f095ce33409',
     'name': {'de': 'Erstes Textfeld:'}}],
   'inputType': 'SINGLELINE'},
  {'name': {'de': 'Zweites Textfeld:'},
   'labels': [],
   'range': [],
   'varnames': ['V2'],
   'codes': [],
   'items': [{'id': 'V01_6553e5564b7d645accf244bfb9a06556',
     'name': {'de': 'Zweites Textfeld:'}}],
   'inputType': 'SINGLELINE'},
  {'name': {'de': 'Drittes Textfeld:'},
   'labels': [],
   'range': [],
   'varnames': ['V3'],
   'codes': [],
   'items': [{'id': 'V01_4f9d0986232055992b5504c6ce424d6f',
     'name': {'de': 'Drittes Textfeld:'}}],
   'inputType': 'SINGLELINE'}]}

### Creating a dataframe

In [11]:
unique_pages = list(set([question["pageId"] for question in questions]))
pages_dict = dict(zip(unique_pages, range(1, len(unique_pages)+1)))


df = pl.DataFrame(
    {
        "text": [q["question"]["de"] for q in questions],
        "id": [q["id"] for q in questions],
        "absolute_position": list(range(1, len(questions) + 1)),
        "relative_section_position": [q["position"] for q in questions],
        "type": [question["type"] for question in questions],
        "number_groups": [len(question["groups"]) for question in questions],
        "page_number": [pages_dict[question["pageId"]] for question in questions],
    }
)
df.head()

text,id,absolute_position,relative_section_position,type,number_groups,page_number
str,i64,i64,i64,str,i64,i64
"""Welche drei Worte verbindest d…",27937500,1,2,"""INPUT""",3,1
"""Wie lange bist du schon bei [U…",27937503,2,3,"""CHOICE""",1,1
"""Wie bist du zu [U25] gekommen?""",27937506,3,4,"""CHOICE""",1,1
"""Nachfolgend findest du einige …",27937509,4,5,"""MATRIX""",1,1
"""Wie geht es dir damit aktuell?…",27937512,5,7,"""CHOICE""",1,1


### Types of Questions:

In [12]:
df["type"].value_counts()

type,count
str,u32
"""INPUT""",7
"""SCALE""",2
"""MATRIX""",1
"""CHOICE""",12


No questions are mandatory

In [13]:
pl.DataFrame([question["mandatory"] for question in questions])["column_0"].value_counts()

column_0,count
bool,u32
false,22


#### Number of Subgroups of questions

In [14]:
df["number_groups"].value_counts().sort("count", descending=True)

number_groups,count
i64,u32
1,17
2,4
3,1


Count = 1 should be easy to handle

### Varnames

In [15]:
pl.DataFrame([len(g["varnames"]) for q in questions for g in q["groups"]]).rename({"column_0": "number_varnames"})["number_varnames"].value_counts()

number_varnames,count
i64,u32
8,1
14,2
1,23
6,1
10,1


In [16]:
questions[0]["groups"][0]["varnames"]

['V1']

## Items

In [17]:
pl.DataFrame([len(g["items"]) for q in questions for g in q["groups"]]).rename({"column_0": "number_items"})["number_items"].value_counts()

number_items,count
i64,u32
14,1
1,27


In [18]:
questions[0]["groups"][0]["items"]

[{'id': 'V01_8ff5376a05a639fce2327f095ce33409',
  'name': {'de': 'Erstes Textfeld:'}}]

### Type by group number

In [19]:
combinations = (
    df.group_by("type")
    .agg(pl.col("number_groups").value_counts())
    .explode("number_groups")
    .unnest("number_groups")
)

print(combinations)

shape: (7, 3)
┌────────┬───────────────┬───────┐
│ type   ┆ number_groups ┆ count │
│ ---    ┆ ---           ┆ ---   │
│ str    ┆ i64           ┆ u32   │
╞════════╪═══════════════╪═══════╡
│ INPUT  ┆ 3             ┆ 1     │
│ INPUT  ┆ 2             ┆ 1     │
│ INPUT  ┆ 1             ┆ 5     │
│ CHOICE ┆ 1             ┆ 9     │
│ CHOICE ┆ 2             ┆ 3     │
│ MATRIX ┆ 1             ┆ 1     │
│ SCALE  ┆ 1             ┆ 2     │
└────────┴───────────────┴───────┘


### Look at groups and items

#### Groups

In [20]:
# Create groups dataframe with question_id
groups_data = []
for q in questions:
    question_id = q["id"]
    for i,group in enumerate(q["groups"]):
        group_id = i
        group_name = group.get("name", {})
        groups_data.append({
            "question_id": question_id,
            "group_id": group_id,
            "group_name_de": group_name.get("de") if isinstance(group_name, dict) else None,
            "number_varnames": len(group.get("varnames", [])),
            "number_items": len(group.get("items", [])),
            "number_labels": len(group.get("labels", [])),
        })

df_groups = pl.DataFrame(groups_data)
df_groups.head()


question_id,group_id,group_name_de,number_varnames,number_items,number_labels
i64,i64,str,i64,i64,i64
27937500,0,"""Erstes Textfeld:""",1,1,0
27937500,1,"""Zweites Textfeld:""",1,1,0
27937500,2,"""Drittes Textfeld:""",1,1,0
27937503,0,null,1,1,5
27937506,0,null,8,1,8


#### Items

In [21]:
items_data = []
for q in questions:
    question_id = q["id"]
    for group in q["groups"]:
        input_type = group.get("inputType", None)
        for item in group.get("items", []):
            item_name = item.get("name", {})
            items_data.append({
                "question_id": question_id,
                "item_id": item.get("id"),
                "item_name_de": item_name.get("de") if isinstance(item_name, dict) else None,
                "input_type": input_type,
            })

df_items = pl.DataFrame(items_data)
df_items.head()


question_id,item_id,item_name_de,input_type
i64,str,str,str
27937500,"""V01_8ff5376a05a639fce2327f095c…","""Erstes Textfeld:""","""SINGLELINE"""
27937500,"""V01_6553e5564b7d645accf244bfb9…","""Zweites Textfeld:""","""SINGLELINE"""
27937500,"""V01_4f9d0986232055992b5504c6ce…","""Drittes Textfeld:""","""SINGLELINE"""
27937503,"""V01_9d2266c406d1b0cbb0f54bec41…",null,null
27937506,"""V01_0dc33bd3ee6bd3abe04b37031b…",null,null


In [22]:
df_items["input_type"].value_counts()

input_type,count
str,u32
"""SINGLELINE""",9
null,28
"""MULTILINE""",1
"""INTEGER""",3


#### Groups: More than one variable associated

In [23]:
df_groups.filter(pl.col("number_varnames")>1)

question_id,group_id,group_name_de,number_varnames,number_items,number_labels
i64,i64,str,i64,i64,i64
27937506,0,null,8,1,8
27937509,0,null,14,14,6
27937521,0,null,14,1,14
27937524,0,null,10,1,10
27937548,0,null,6,1,6


One item but multiple variable names (set of boolean question)

In [24]:
df_groups.filter(pl.col("question_id") == 27937521)

question_id,group_id,group_name_de,number_varnames,number_items,number_labels
i64,i64,str,i64,i64,i64
27937521,0,null,14,1,14
27937521,1,"""Anderes""",1,1,0


But question has multiple groups

In [25]:
df.filter(pl.col("id") == 27937521)

text,id,absolute_position,relative_section_position,type,number_groups,page_number
str,i64,i64,i64,str,i64,i64
"""Warum engagierst du dich bei […",27937521,8,1,"""CHOICE""",2,2


The second one is an open input type

In [26]:
df_items.filter(pl.col("question_id") == 27937521)

question_id,item_id,item_name_de,input_type
i64,str,str,str
27937521,"""V01_4bd0b717716e648690c1713cdf…",null,null
27937521,"""V01_30a569a5c44e6c5b773fd26251…",null,"""SINGLELINE"""


#### Groups: More than one item

In [27]:
df_groups.filter(pl.col("number_items")>1)

question_id,group_id,group_name_de,number_varnames,number_items,number_labels
i64,i64,str,i64,i64,i64
27937509,0,null,14,14,6


##### Multiple Items means its a matrix and each item equals a variable

In [28]:
df_items.filter(pl.col("question_id") == 27937509)

question_id,item_id,item_name_de,input_type
i64,str,str,str
27937509,"""V01_244c5b063ac9501b821a52eae3…","""Ich habe Bedenken, ob ich den …",null
27937509,"""V01_00c4a147de0511441bd9301c5a…","""Ich merke, dass ich im Laufe d…",null
27937509,"""V01_a14946f573506698e725fbd0da…","""Bei [U25] kann ich mich auspro…",null
27937509,"""V01_01fcfce935537bf09346dd7681…","""Ich habe das Gefühl, dass ich …",null
27937509,"""V01_cdcaf7020652995e65b68f27aa…","""Es war eine gute Entscheidung …",null
…,…,…,…
27937509,"""V01_f78a72528d5610e26f392e2f69…","""Durch mein Engagement bei [U25…",null
27937509,"""V01_6903bf46ad0de46133ed26f3c5…","""Ich gehe aufgrund der Erfahrun…",null
27937509,"""V01_6e8ff276bf9178e68c4d5a35c2…","""Falls ich einmal nicht weiterk…",null


In [29]:
str(df.filter(pl.col("id") == 27937509)["text"][0])

'Nachfolgend findest du einige Aussagen zu [U25]. Bitte bewerte diese auf der Skala:'

In [30]:
str(df.filter(pl.col("id") == 27937509)["type"][0])

'MATRIX'

In [31]:
df_items.filter(pl.col("question_id") == 27937521)

question_id,item_id,item_name_de,input_type
i64,str,str,str
27937521,"""V01_4bd0b717716e648690c1713cdf…",null,null
27937521,"""V01_30a569a5c44e6c5b773fd26251…",null,"""SINGLELINE"""


#### Groups: More than one label

In [32]:
df_groups.filter(pl.col("number_labels")>1).sort("number_labels", descending=True)

question_id,group_id,group_name_de,number_varnames,number_items,number_labels
i64,i64,str,i64,i64,i64
27937521,0,null,14,1,14
27937536,0,null,1,1,11
27937524,0,null,10,1,10
27937506,0,null,8,1,8
27937509,0,null,14,14,6
…,…,…,…,…,…
27937530,0,null,1,1,5
27937515,0,null,1,1,4
27937539,0,null,1,1,3


14 Items and 6 Labels? Labels and Variables are equal though -> 

In [33]:
df.filter(pl.col("id") == 27937509)

text,id,absolute_position,relative_section_position,type,number_groups,page_number
str,i64,i64,i64,str,i64,i64
"""Nachfolgend findest du einige …",27937509,4,5,"""MATRIX""",1,1


In [ ]:
str(df.filter(pl.col("id") == 27937509)["text"][0])

'shape: (1, 7)\n┌────────────────┬──────────┬───────────────┬───────────────┬────────┬───────────────┬─────────────┐\n│ text           ┆ id       ┆ absolute_posi ┆ relative_sect ┆ type   ┆ number_groups ┆ page_number │\n│ ---            ┆ ---      ┆ tion          ┆ ion_position  ┆ ---    ┆ ---           ┆ ---         │\n│ str            ┆ i64      ┆ ---           ┆ ---           ┆ str    ┆ i64           ┆ i64         │\n│                ┆          ┆ i64           ┆ i64           ┆        ┆               ┆             │\n╞════════════════╪══════════╪═══════════════╪═══════════════╪════════╪═══════════════╪═════════════╡\n│ Dein           ┆ 27937548 ┆ 16            ┆ 2             ┆ CHOICE ┆ 2             ┆ 3           │\n│ Geschlecht:    ┆          ┆               ┆               ┆        ┆               ┆             │\n└────────────────┴──────────┴───────────────┴───────────────┴────────┴───────────────┴─────────────┘'

In [80]:
questions[6]

{'id': 27937575,
 'pollId': 1850964,
 'pageId': 8134161,
 'pagePosition': 0,
 'position': 6,
 'question': {'de': 'Wie viel Zeit verbringst du pro Monat mit [U25]?'},
 'description': {'de': 'Wir wissen, dass dies schwanken kann - daher bitten wir dich eine Spanne anzugeben - wie viele Stunden sind es mindestens und wie viele maximal?'},
 'anonym': False,
 'mandatory': False,
 'type': 'INPUT',
 'groups': [{'name': {'de': 'Mindeste Stundenanzahl pro Monat:'},
   'labels': [],
   'range': [0, 1000000, 1],
   'varnames': ['V13'],
   'codes': [],
   'items': [{'id': 'V01_8a4c847bd49a0c3b0bc0c3ebd0f80c20',
     'name': {'de': 'Mindeste Stundenanzahl pro Monat:'}}],
   'inputType': 'INTEGER'},
  {'name': {'de': 'Maximale Stundenanzahl pro Monat:'},
   'labels': [],
   'range': [0, 1000000, 1],
   'varnames': ['V72'],
   'codes': [],
   'items': [{'id': 'V01_4abaeadfc3bbe1bc14338fd1d3ff4ded',
     'name': {'de': 'Maximale Stundenanzahl pro Monat:'}}],
   'inputType': 'INTEGER'}]}

Labels do not equals variables?

In [36]:
df_groups.filter(pl.col("number_labels") != pl.col("number_varnames")).sort("number_labels", descending=True)

question_id,group_id,group_name_de,number_varnames,number_items,number_labels
i64,i64,str,i64,i64,i64
27937536,0,null,1,1,11
27937509,0,null,14,14,6
27937554,0,null,1,1,6
27937503,0,null,1,1,5
27937512,0,null,1,1,5
…,…,…,…,…,…
27937545,0,null,1,1,0
27937548,1,null,1,1,0
27937551,0,null,1,1,0


In [37]:
df.filter(pl.col("id") == 27937536)

text,id,absolute_position,relative_section_position,type,number_groups,page_number
str,i64,i64,i64,str,i64,i64
"""Zu welchem Standort gehörst du…",27937536,15,1,"""CHOICE""",1,3


When theres is one variable but multiple lables: single choice?

In [83]:
df.filter(pl.col("id") == 27937548)

text,id,absolute_position,relative_section_position,type,number_groups,page_number
str,i64,i64,i64,str,i64,i64
"""Dein Geschlecht:""",27937548,16,2,"""CHOICE""",2,3


In [84]:
questions[15]

{'id': 27937548,
 'pollId': 1850964,
 'pageId': 8134167,
 'pagePosition': 2,
 'position': 2,
 'question': {'de': 'Dein Geschlecht:'},
 'description': {},
 'anonym': False,
 'mandatory': False,
 'type': 'CHOICE',
 'groups': [{'name': [],
   'labels': [{'de': 'nicht-binär'},
    {'de': 'weiblich'},
    {'de': 'männlich'},
    {'de': 'kein Geschlecht'},
    [],
    {'de': 'keine Angabe'}],
   'range': [],
   'varnames': ['V59', 'V67', 'V68', 'V69', 'V70', 'V71'],
   'codes': ['1', '1', '1', '1', '1', '1'],
   'items': [{'id': 'V01_c9fa4a41056e28a34f8ceb3d4e3a7641', 'name': []}]},
  {'name': [],
   'labels': [],
   'range': [],
   'varnames': ['V70.1'],
   'codes': ['1'],
   'items': [{'id': 'V01_30314feba570c50d4539d91e9623ae41', 'name': []}],
   'inputType': 'SINGLELINE'}]}

## Results

In [40]:
response = requests.get(f"{BASE_URL}/polls/{poll_id}/questions/{27937521}/results", headers=headers)
response.raise_for_status()
results = response.json()
results

{'questionId': 27937521,
 'question': '<strong><span style="font-size:24px;">Warum engagierst du dich bei [U25] ?</span></strong>',
 'description': '(Mehrfachnennung möglich)',
 'summary': {'responses': 86,
  'seen': 86,
  'answered': 79,
  'ignored': 7,
  'noAnswer': None,
  'answers': 548},
 'groups': [{'name': '',
   'labels': ['Ich möchte anderen helfen.',
    'Ich möchte anderen weitergeben, was ich selbst in psychischen Krisen gelernt habe.',
    'Ich finde das Thema Suizidprävention gesellschaftlich wichtig.',
    'Ich möchte mit meinem Engagement etwas Sinnvolles tun.',
    'Ich möchte mich persönlich weiterentwickeln.',
    'Ich möchte im Bereich psychische Gesundheit dazulernen.',
    'Ich sehe das Engagement als Möglichkeit zur Berufsorientierung.',
    'Ich sehe mein Engagement bei [U25] als relevante Zusatzqualifikation im Lebenslauf.',
    'Ich schätze bei [U25] soziale Kontakte zu Peers und Koordinator*innen.',
    'Anderes',
    'Ich fühle mich zur Ausübung eines ehrena

In [77]:
questions[1]

{'id': 27937503,
 'pollId': 1850964,
 'pageId': 8134161,
 'pagePosition': 0,
 'position': 3,
 'question': {'de': 'Wie lange bist du schon bei [U25]?'},
 'description': {},
 'anonym': False,
 'mandatory': False,
 'type': 'CHOICE',
 'groups': [{'name': [],
   'labels': [{'de': 'weniger als 1 Jahr'},
    {'de': '1-2 Jahre'},
    {'de': '3-4 Jahre'},
    {'de': '5-6 Jahre'},
    {'de': 'mehr als 6 Jahre'}],
   'range': [],
   'varnames': ['V4'],
   'codes': ['1', '2', '3', '4', '5'],
   'items': [{'id': 'V01_9d2266c406d1b0cbb0f54bec41846dfd', 'name': []}]}]}

In [41]:
# results[0]["summary"]

In [42]:
# results[0]["groups"][0]

## Legacy

In [43]:
response = requests.get(f"{BASE_URL}/polls/{poll_id}/legacyResults", headers=headers)
response.raise_for_status()
legacy = response.json()

In [44]:
leg_df = pl.read_csv(io.StringIO(legacy["data"]))
leg_df.head()

vID,vANONYM,vCOMPLETED,vFINISHED,vDURATION,vQUOTE,vLANG,vSTART,vEND,vRUNTIME,vPAGETIME1,vPAGETIME2,vPAGETIME3,vDATE,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V27.C1,V28.C1,V29.C1,V30.C1,V31.C1,V32.C1,V33.C1,V34.C1,V35.C1,V36.C1,V37.C1,…,V24,V25,V26,V40,V41,V42,V42.1,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V52.1,V53,V58,V55,V56,V57,V54,V59,V67,V68,V69,V70,V71,V70.1,V60,V61,V62,V63,V64,V65
i64,i64,i64,i64,f64,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,1,0,0,347.67,""" 0%""","""de""","""2025-08-08""","""""",""" 00:05:48""",172,175,0,"""2025-08-08""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",…,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
2,1,0,0,14988.84,""" 0%""","""de""","""2025-08-11""","""""",""" 04:09:49""",685,7,14297,"""2025-08-11""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",…,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
3,1,1,1,239.53,"""95.45%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:04:00""",126,85,28,"""2025-08-12""","""Zusammenhalt""","""Unterstützung""","""Wertschätzung""","""3""","""""","""""","""""","""""","""1""","""""","""""","""""","""3""","""5""","""6""","""6""","""4""","""6""","""6""","""6""","""6""","""6""","""4""",…,"""1""","""""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""1""","""1""","""""","""1""","""1""","""1""","""""","""""","""2""","""3""","""8""","""Wenn die Person zu große eigen…","""Alles gut""","""8""","""""","""1""","""""","""""","""""","""""","""""","""23""","""3""","""1""","""Nein :)""","""5""",""""""
4,1,1,1,917.07,"""90.91%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:15:18""",752,136,30,"""2025-08-12""","""Wärme""","""Herzlichkeit""","""Angenommen werden""","""2""","""""","""""","""""","""""","""1""","""1""","""""","""""","""1""","""6""","""6""","""6""","""5""","""6""","""6""","""6""","""6""","""6""","""5""",…,"""1""","""""","""1""","""""","""1""","""""","""""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""""","""""","""1""","""2""","""10""","""Kein U25 Standort in der Nähe …","""Mehr Fortbildungen""","""8""","""""","""1""","""""","""""","""""","""""","""""","""23""","""3""","""1""","""""","""5""",""""""
5,1,1,1,938.24,"""77.27%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:15:39""",492,354,93,"""2025-08-12""","""""","""""","""""","""2""","""""","""""","""""","""""","""1""","""""","""""","""""","""4""","""5""","""5""","""6""","""4""","""5""","""5""","""3""","""3""","""4""","""5""",…,"""""","""1""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""1""","""""","""""","""""","""1""","""""","""""","""""","""3""","""3""","""7""","""""","""""","""8""","""""","""1""","""""","""""","""""","""""","""""","""21""","""3""","""1""","""""","""5""",""""""


This data format is not only flat with regards to the participants but groups are unnested as well.

In [75]:
leg_df.filter(
    (pl.col("vCOMPLETED") != 0) &
    ~pl.all_horizontal(
        cs.matches("^V\\d").fill_null("").cast(pl.String, strict=False) == ""
    )
)

vID,vANONYM,vCOMPLETED,vFINISHED,vDURATION,vQUOTE,vLANG,vSTART,vEND,vRUNTIME,vPAGETIME1,vPAGETIME2,vPAGETIME3,vDATE,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V27.C1,V28.C1,V29.C1,V30.C1,V31.C1,V32.C1,V33.C1,V34.C1,V35.C1,V36.C1,V37.C1,…,V24,V25,V26,V40,V41,V42,V42.1,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V52.1,V53,V58,V55,V56,V57,V54,V59,V67,V68,V69,V70,V71,V70.1,V60,V61,V62,V63,V64,V65
i64,i64,i64,i64,f64,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
3,1,1,1,239.53,"""95.45%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:04:00""",126,85,28,"""2025-08-12""","""Zusammenhalt""","""Unterstützung""","""Wertschätzung""","""3""","""""","""""","""""","""""","""1""","""""","""""","""""","""3""","""5""","""6""","""6""","""4""","""6""","""6""","""6""","""6""","""6""","""4""",…,"""1""","""""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""1""","""1""","""""","""1""","""1""","""1""","""""","""""","""2""","""3""","""8""","""Wenn die Person zu große eigen…","""Alles gut""","""8""","""""","""1""","""""","""""","""""","""""","""""","""23""","""3""","""1""","""Nein :)""","""5""",""""""
4,1,1,1,917.07,"""90.91%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:15:18""",752,136,30,"""2025-08-12""","""Wärme""","""Herzlichkeit""","""Angenommen werden""","""2""","""""","""""","""""","""""","""1""","""1""","""""","""""","""1""","""6""","""6""","""6""","""5""","""6""","""6""","""6""","""6""","""6""","""5""",…,"""1""","""""","""1""","""""","""1""","""""","""""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""""","""""","""1""","""2""","""10""","""Kein U25 Standort in der Nähe …","""Mehr Fortbildungen""","""8""","""""","""1""","""""","""""","""""","""""","""""","""23""","""3""","""1""","""""","""5""",""""""
5,1,1,1,938.24,"""77.27%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:15:39""",492,354,93,"""2025-08-12""","""""","""""","""""","""2""","""""","""""","""""","""""","""1""","""""","""""","""""","""4""","""5""","""5""","""6""","""4""","""5""","""5""","""3""","""3""","""4""","""5""",…,"""""","""1""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""1""","""""","""""","""""","""1""","""""","""""","""""","""3""","""3""","""7""","""""","""""","""8""","""""","""1""","""""","""""","""""","""""","""""","""21""","""3""","""1""","""""","""5""",""""""
6,1,1,1,226.5,"""86.36%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:03:47""",139,71,17,"""2025-08-12""","""Community""","""Hilfe""","""Verständnis""","""3""","""""","""""","""1""","""""","""""","""""","""""","""""","""1""","""6""","""6""","""6""","""5""","""5""","""6""","""6""","""6""","""6""","""5""",…,"""1""","""""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""""","""1""","""1""","""""","""""","""1""","""""","""""","""2""","""2""","""9""","""Wenn man sich nicht mit dem Th…","""""","""6""","""""","""1""","""""","""""","""""","""""","""""","""24""","""4""","""1""","""""","""5""",""""""
7,1,1,1,838.31,"""100%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:13:59""",651,136,51,"""2025-08-12""","""Unterstützung""","""Akzeptanz""","""Sicherheit""","""4""","""1""","""""","""""","""""","""""","""""","""""","""""","""2""","""6""","""5""","""6""","""4""","""5""","""6""","""5""","""6""","""5""","""5""",…,"""1""","""""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""1""","""""","""""","""1""","""1""","""""","""""","""""","""2""","""2""","""8""","""Persönliche Belastung mit dem …","""Mehr Sichtbarkeit für ein wich…","""7""","""""","""""","""1""","""""","""""","""""","""""","""24""","""3""","""2""","""/""","""5""","""/"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
78,1,1,1,594

In [62]:
leg_df.filter((pl.col("vCOMPLETED") == True) & (pl.col("vFINISHED") == True))

vID,vANONYM,vCOMPLETED,vFINISHED,vDURATION,vQUOTE,vLANG,vSTART,vEND,vRUNTIME,vPAGETIME1,vPAGETIME2,vPAGETIME3,vDATE,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V27.C1,V28.C1,V29.C1,V30.C1,V31.C1,V32.C1,V33.C1,V34.C1,V35.C1,V36.C1,V37.C1,…,V24,V25,V26,V40,V41,V42,V42.1,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V52.1,V53,V58,V55,V56,V57,V54,V59,V67,V68,V69,V70,V71,V70.1,V60,V61,V62,V63,V64,V65
i64,i64,i64,i64,f64,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
3,1,1,1,239.53,"""95.45%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:04:00""",126,85,28,"""2025-08-12""","""Zusammenhalt""","""Unterstützung""","""Wertschätzung""","""3""","""""","""""","""""","""""","""1""","""""","""""","""""","""3""","""5""","""6""","""6""","""4""","""6""","""6""","""6""","""6""","""6""","""4""",…,"""1""","""""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""1""","""1""","""""","""1""","""1""","""1""","""""","""""","""2""","""3""","""8""","""Wenn die Person zu große eigen…","""Alles gut""","""8""","""""","""1""","""""","""""","""""","""""","""""","""23""","""3""","""1""","""Nein :)""","""5""",""""""
4,1,1,1,917.07,"""90.91%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:15:18""",752,136,30,"""2025-08-12""","""Wärme""","""Herzlichkeit""","""Angenommen werden""","""2""","""""","""""","""""","""""","""1""","""1""","""""","""""","""1""","""6""","""6""","""6""","""5""","""6""","""6""","""6""","""6""","""6""","""5""",…,"""1""","""""","""1""","""""","""1""","""""","""""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""""","""""","""1""","""2""","""10""","""Kein U25 Standort in der Nähe …","""Mehr Fortbildungen""","""8""","""""","""1""","""""","""""","""""","""""","""""","""23""","""3""","""1""","""""","""5""",""""""
5,1,1,1,938.24,"""77.27%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:15:39""",492,354,93,"""2025-08-12""","""""","""""","""""","""2""","""""","""""","""""","""""","""1""","""""","""""","""""","""4""","""5""","""5""","""6""","""4""","""5""","""5""","""3""","""3""","""4""","""5""",…,"""""","""1""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""1""","""""","""""","""""","""1""","""""","""""","""""","""3""","""3""","""7""","""""","""""","""8""","""""","""1""","""""","""""","""""","""""","""""","""21""","""3""","""1""","""""","""5""",""""""
6,1,1,1,226.5,"""86.36%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:03:47""",139,71,17,"""2025-08-12""","""Community""","""Hilfe""","""Verständnis""","""3""","""""","""""","""1""","""""","""""","""""","""""","""""","""1""","""6""","""6""","""6""","""5""","""5""","""6""","""6""","""6""","""6""","""5""",…,"""1""","""""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""""","""1""","""1""","""""","""""","""1""","""""","""""","""2""","""2""","""9""","""Wenn man sich nicht mit dem Th…","""""","""6""","""""","""1""","""""","""""","""""","""""","""""","""24""","""4""","""1""","""""","""5""",""""""
7,1,1,1,838.31,"""100%""","""de""","""2025-08-12""","""2025-08-12""",""" 00:13:59""",651,136,51,"""2025-08-12""","""Unterstützung""","""Akzeptanz""","""Sicherheit""","""4""","""1""","""""","""""","""""","""""","""""","""""","""""","""2""","""6""","""5""","""6""","""4""","""5""","""6""","""5""","""6""","""5""","""5""",…,"""1""","""""","""1""","""""","""""","""""","""""","""1""","""1""","""1""","""1""","""""","""""","""1""","""1""","""""","""""","""""","""2""","""2""","""8""","""Persönliche Belastung mit dem …","""Mehr Sichtbarkeit für ein wich…","""7""","""""","""""","""1""","""""","""""","""""","""""","""24""","""3""","""2""","""/""","""5""","""/"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
79,1,1,1,647

In [76]:
leg_df.schema

Schema([('vID', Int64),
        ('vANONYM', Int64),
        ('vCOMPLETED', Int64),
        ('vFINISHED', Int64),
        ('vDURATION', Float64),
        ('vQUOTE', String),
        ('vLANG', String),
        ('vSTART', String),
        ('vEND', String),
        ('vRUNTIME', String),
        ('vPAGETIME1', Int64),
        ('vPAGETIME2', Int64),
        ('vPAGETIME3', Int64),
        ('vDATE', String),
        ('V1', String),
        ('V2', String),
        ('V3', String),
        ('V4', String),
        ('V5', String),
        ('V6', String),
        ('V7', String),
        ('V8', String),
        ('V9', String),
        ('V10', String),
        ('V11', String),
        ('V12', String),
        ('V27.C1', String),
        ('V28.C1', String),
        ('V29.C1', String),
        ('V30.C1', String),
        ('V31.C1', String),
        ('V32.C1', String),
        ('V33.C1', String),
        ('V34.C1', String),
        ('V35.C1', String),
        ('V36.C1', String),
        ('V37.C1', String